# Project Design - Detection Demonstration


In [ ]:
# Mout your Google Drive to this notebook
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

In [ ]:
%cd "/content/drive/MyDrive"

In [ ]:
!rm -rf ./detection

In [ ]:
!mkdir -p ./detection

In [ ]:
!git clone https://<username>:<token>@github.com/cpe-pd/rpi-camera.git ./detection/rpi-camera

In [ ]:
!git clone https://github.com/WongKinYiu/yolor.git ./detection/rpi-camera/yolor

In [ ]:
!mkdir -p ./detection/sample

In [ ]:
# Put images in the ./detection/sample

In [ ]:
!pip install -r ./detection/rpi-camera/yolor/requirements.txt

In [ ]:
!pip install face_recognition

In [ ]:
!pip install paho-mqtt

In [ ]:
!pip install gdown

In [ ]:
%cd ./detection/rpi-camera

In [ ]:
%cd data

In [ ]:
!gdown 1-6HScC2uanGPQd3BDOPj84DE3yyxEand

In [ ]:
%cd ..

In [ ]:
!chmod +x ./scripts/linux/install.sh

In [ ]:
!./scripts/linux/install.sh

In [ ]:
from google.colab.patches import cv2_imshow
from yolor.utils.datasets import letterbox
from configparser import ConfigParser
from src.detection import Detection
from src.utils import checkLatestWeights
from base64 import b64encode, b64decode
import numpy as np
import cv2, os, torch, glob

In [ ]:
def newDimensionAndRGBFormat(image: np.ndarray) -> np.ndarray:
  result = image.copy()
  result = letterbox(result, auto=True)[0]
  result = np.expand_dims(result, axis=0)
  result = result.transpose(0, 3, 1, 2)
  return result
  
def readImage(image_file: str) -> np.ndarray:
  if not os.path.exists(image_file):
    raise Exception(f"[ERROR] '{image_file}' does not exist")
  image = cv2.imread(image_file)
  return image

def detectPPE(det: Detection, image_file: str):
  image = readImage(image_file)
  rgb_image = newDimensionAndRGBFormat(image)
  result = det.detect(rgb_image, image)
  return result

def decodeEncodedImage(encodedStr: str):
  buffer = b64decode(encodedStr)
  npimg = np.frombuffer(buffer, dtype=np.uint8)
  image = cv2.imdecode(npimg, 1)
  image = cv2.resize(image, (800, 800), interpolation=cv2.INTER_NEAREST)
  return image

In [ ]:
checkLatestWeights()
cfg = ConfigParser()
cfg.read("./cfg/app.cfg")
det = Detection(cfg)

In [ ]:
img_files = glob.glob(os.path.join("..", "sample", "*"))

In [ ]:
with torch.no_grad():
  for img_file in img_files:
    try:
      result = detectPPE(det, img_file)
      image = decodeEncodedImage(result["image"])
      cv2_imshow(image)
    except Exception as e:
      print(f"{e}")